In [1]:
import torch
import pickle
import pandas as pd
from pathlib import Path
import sys

sys.path.append("Users/erogullari/Desktop/Workspace/cav-disentanglement/")

In [ ]:
with open("/home/erogullari/Workspace/cav-disentanglement/results/model_correction/vgg16/features.28/pattern_cav/alpha0.1_lr0.001/metrics/all_metrics.pkl", "rb") as f:
    all_metrics = pickle.load(f)
with open("/home/erogullari/Workspace/cav-disentanglement/results/model_correction/vgg16/features.28/pattern_cav/alpha0.1_lr0.001/metrics/accuracy_metrics.pkl", "rb") as f:
    accuracy_metrics = pickle.load(f)


df = pd.DataFrame(all_metrics)
start = 20
df[start:start+20]

,Metric,Value,Model
20,test_acc_Non-Blond_Hair_attacked,0.772261,Vanilla
21,test_acc_Blond_Hair_attacked,0.772261,Vanilla
22,test_accuracy_clean,0.952369,Vanilla
23,test_accuracy_standard_err_clean,0.001496,Vanilla
24,test_f1_clean,0.906644,Vanilla
25,test_AUC_Non-Blond_Hair_clean,0.981652,Vanilla
26,test_AUC_Blond_Hair_clean,0.981569,Vanilla
27,test_fpr_Non-Blond_Hair_clean,0.150448,Vanilla
28,test_fpr_Blond_Hair_clean,0.029683,Vanilla
29,test_fnr_Non-Blond_Hair_clean,0.029683,Vanilla


In [4]:
with open("/home/erogullari/Workspace/cav-disentanglement/results/model_correction/vgg16/features.28/pattern_cav/alpha0.01_lr0.001_opt/metrics/selected_metrics.pkl", "rb") as f:
    selected_metrics = pickle.load(f)
selected_metrics.keys()
df_selected = pd.DataFrame(selected_metrics)

In [5]:
df_selected

,Metric,Value,Model,Category,Metric Type
11,test_accuracy_attacked,0.774951,Vanilla,attacked,test_accuracy
16,test_fpr_0_attacked,0.038922,Vanilla,attacked,test_fpr_0
17,test_fpr_1_attacked,0.261404,Vanilla,attacked,test_fpr
18,test_fnr_0_attacked,0.261404,Vanilla,attacked,test_fnr_0
19,test_fnr_1_attacked,0.038922,Vanilla,attacked,test_fnr
22,test_accuracy_clean,0.964286,Vanilla,clean,test_accuracy
27,test_fpr_0_clean,0.116766,Vanilla,clean,test_fpr_0
28,test_fpr_1_clean,0.019883,Vanilla,clean,test_fpr
29,test_fnr_0_clean,0.019883,Vanilla,clean,test_fnr_0
30,test_fnr_1_clean,0.116766,Vanilla,clean,test_fnr


In [2]:
scavs_baseline = torch.load("/Users/erogullari/Desktop/Workspace/cav-disentanglement/DELETE/clarc/scavs_baseline.pt")
scavs_baseline.shape

torch.Size([42, 512])

In [19]:
vars = torch.load("/Users/erogullari/Desktop/Workspace/cav-disentanglement/variables/celeba_attacked/vgg16/features.28.pth")
encs = vars["encs"]
labels = vars["labels"]
encs.shape, labels.shape

(torch.Size([20438, 512]), torch.Size([20438, 42]))

In [10]:
from typing import Tuple

def compute_cavs(vecs: torch.Tensor, targets: torch.Tensor, type: str = "pattern_cav", normalize: bool = True) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    Computes all concept activation vectors (CAVs) together for a set of vectors and targets.

    :param vecs:    torch.Tensor of shape (n_samples, n_features)
    :param targets: torch.Tensor of shape (n_samples, n_concepts)
    :param type:    str, type of CAV to compute. One of ["pattern_cav", "multi_cav"]
    :param normalize: bool, whether to normalize the CAVs to unit length
    :return:        tuple of (cavs, bias)
    """
    mu_x = vecs.mean(dim=0) 
    mu_y = targets.mean(dim=0)  
    X_centered = vecs - mu_x.unsqueeze(0)                           # (n_samples, n_features)
    Y_centered = targets - mu_y.unsqueeze(0)                        # (n_samples, n_concepts)
    n_samples = vecs.shape[0]

    if type == "pattern_cav":
        covars = (Y_centered.T @ X_centered) / (n_samples - 1)      # (n_concepts, n_features)
        vars = torch.sum(Y_centered**2, dim=0) / (n_samples - 1)    # (n_concepts,)
        inv_vars = torch.zeros_like(vars)
        nz = vars > 0
        inv_vars[nz] = 1.0 / vars[nz]
        cavs = covars * inv_vars.unsqueeze(1)                       # (n_concepts, n_features)

        if normalize:
            cavs /= torch.norm(cavs, dim=1, keepdim=True)
        
        # For zero-variance rows, best constant fit is mu_x
        bias = mu_x.unsqueeze(0) - mu_y.unsqueeze(1) * cavs         # (n_concepts, n_features)
        if (~nz).any():
            bias[(~nz), :] = mu_x.unsqueeze(0)

    elif type == "multi_cav":
        y_aug = Y_centered.T @ Y_centered
        cavs = torch.linalg.pinv(y_aug) @ (Y_centered.T @ X_centered)   # (n_concepts, n_features)
        if normalize:
            cavs /= torch.norm(cavs, dim=1, keepdim=True)
        bias = (mu_x - cavs.T @ mu_y).unsqueeze(0)                      # (1, n_features)

    else:
        raise KeyError(f"Unknown CAV type: {type}")

    return cavs, bias

scavs, _ = compute_cavs(encs, labels, type="pattern_cav", normalize=True)
scavs.shape

torch.Size([42, 512])

In [13]:
scavs

tensor([[ 0.0018,  0.0126,  0.0024,  ...,  0.0584,  0.0148,  0.0324],
        [ 0.0203,  0.0332, -0.0274,  ..., -0.0626,  0.0031, -0.0491],
        [ 0.0078,  0.0445, -0.0424,  ..., -0.0329, -0.0159, -0.0500],
        ...,
        [-0.0505,  0.0303, -0.0784,  ..., -0.0584,  0.0160, -0.0398],
        [ 0.0744,  0.0174,  0.0076,  ...,  0.0380,  0.0150, -0.0089],
        [ 0.0596,  0.0158,  0.0064,  ...,  0.0258,  0.0167, -0.0043]])

In [15]:
torch.linalg.norm(scavs - scavs_baseline, dim=1)

tensor([0.0974, 0.0809, 0.0961, 0.1013, 0.0991, 0.0939, 0.1364, 0.0856, 0.0757,
        0.0557, 0.1030, 0.1134, 0.0920, 0.1066, 0.1113, 0.0816, 0.0881, 0.0944,
        0.0827, 0.0831, 0.0833, 0.0977, 0.0937, 0.2059, 0.0824, 0.1376, 0.1473,
        0.0909, 0.0964, 0.0844, 0.0799, 0.0980, 0.1505, 0.1088, 0.0859, 0.0769,
        0.0803, 0.0860, 0.1587, 0.0904, 0.0355, 0.0507])

In [2]:
activations_train = torch.load("/Users/erogullari/Desktop/Workspace/cav-disentanglement/DELETE/clarc/activations_train.pth")
labels_train = torch.load("/Users/erogullari/Desktop/Workspace/cav-disentanglement/DELETE/clarc/labels_train.pth")
activations_train.shape, labels_train.shape

(torch.Size([16214, 512]), torch.Size([16214, 42]))

In [34]:
id = 1
print(torch.equal(labels[id], labels_train[id]))
print(torch.equal(activations_train[id], encs[id]))

True
False


In [4]:
torch.save({
    "encs": activations_train,
    "labels": labels_train.float()
}, "/Users/erogullari/Desktop/Workspace/cav-disentanglement/variables/celeba_attacked/vgg16/features.28.pth")